In [2]:
from reasoning_engines.langchain_llm_wrappers import QuickAzureChatOpenAI
from reasoning_engines.langchain_llm_wrappers import QuickOpenAIClient
from reasoning_engines.langchain_llm_wrappers import QuickAzureOpenAIClient
openai_client = QuickOpenAIClient()
azure_openai_client = QuickAzureOpenAIClient('1106')
llm = QuickAzureChatOpenAI('1106')
# openai_response = openai_client.chat_completions_create(query='How is the weather in Seoul, Bangkok and LA?', tools=openai_functions)
# azure_openai_client.chat_completions_create(query='hello')
from tools.get_tools import DefaultSchemasTools
default_schemas_and_tools = DefaultSchemasTools(azure_gpt_version='1106')
schemas_and_tools = default_schemas_and_tools.schemas_and_tools()
schemas = default_schemas_and_tools.schemas()
tools = default_schemas_and_tools.tools()
tool_dictionary = default_schemas_and_tools.tool_dictionary()
from langchain.utils.openai_functions import convert_pydantic_to_openai_tool
openai_functions = [convert_pydantic_to_openai_tool(x) for x in schemas]


from utils.wrappers import retry
from openai import RateLimitError
@retry(allowed_exceptions=(RateLimitError,))
def get_function_response(function_name, function_args):
    return tool_dictionary[function_name].run(**function_args)


from agents.parallel_func_calling_agent import OpenAIParallelFuntionCallingAgent
from chains.qa_evaluators import CustomQAEvaluator

parallel_calling_agent = OpenAIParallelFuntionCallingAgent(
    reasoninig_engine = QuickAzureChatOpenAI('1106'),
    base_prompt = 'jet-taekyo-lee/parallel-function-calling-agent',
    schemas_and_tools = DefaultSchemasTools(azure_gpt_version='1106'),
    evaluator = CustomQAEvaluator(llm=QuickAzureChatOpenAI('1106')),
    action_word = 'Tool invocations in parallel',
    use_chat_completion_api = True,
    azure_apenai_client = QuickAzureOpenAIClient('1106'),
    horizon = 3
)

query="How old are the current Korean president and Joe Biden?"
# query="How old are the current Korean president?"
query="What is the summation of the temperatures of Seoul, Tokyo, and New york?"
reference="Joe Biden is 81 years old and the current Korean president, Yoon Suk Yeol, is 62 years old."
parallel_calling_agent.clear_logs()
parallel_calling_agent.run_agent_trials(num_trials=1, 
query=query)
# reference=reference)

# import time
# for _ in range(5):
#     for s in range(60, 0, -1):
#         print(s, end=' ')
#         time.sleep(1)   
#     parallel_calling_agent.clear_logs()

#     parallel_calling_agent.agent_step(query)
    # parallel_calling_agent.run_agent_trials(num_trials=3, 
    # query=query,
    # reference=reference)


[INFO] Query: What is the summation of the temperatures of Seoul, Tokyo, and New york?
[INFO] Trial 1
[INFO] Tool invocations in parallel (step 1): [Tool 1-> OpenWeatherMap(IANA_timezone=Asia/Seoul), Tool 2-> OpenWeatherMap(IANA_timezone=Asia/Tokyo), Tool 3-> OpenWeatherMap(IANA_timezone=America/New_York)]
[INFO] Results in parallel (step 1): [Tool 1-> timezone: Seoul, current time(year-month-day): 2023-11-29, temperature: 1.2300000000000182, humidity: 53, wind_speed: 6.22, weather_summary: Expect a day of partly cloudy with snow., Tool 2-> timezone: Tokyo, current time(year-month-day): 2023-11-29, temperature: 13.520000000000039, humidity: 35, wind_speed: 5.51, weather_summary: You can expect clear sky in the morning, with partly cloudy in the afternoon., Tool 3-> timezone: New York County, current time(year-month-day): 2023-11-29, temperature: 1.0500000000000114, humidity: 42, wind_speed: 7.23, weather_summary: The day will start with clear sky through the late morning hours, transit